In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet("breastcancer.parquet")

In [ ]:
len(df)

In [ ]:
df

In [ ]:
df = df[df["time"].dt.year >= 2020].copy()

# Sentence splitting

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm.auto import tqdm
df["sentences"] = None

df["nouns"] = None
df["adjectives"] = None
df["verbs"] = None
df["lemmas"] = None

df["entities"] = None
for i, row in tqdm(df.iterrows(), total=len(df)):

    nouns = []
    adjectives = []
    verbs = []
    lemmas = []
    nav = []

    doc = nlp(row["title"] + ".\n" + row["text"])
    for token in doc:
        lemmas.append(token.lemma_)
        # noun
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            nouns.append(token.lemma_)
            nav.append(token.lemma_)
        # adjective
        if token.pos_ == "ADJ" or token.pos_ == "ADV":
            adjectives.append(token.lemma_)
            nav.append(token.lemma_)
        # verb
        if token.pos_ == "VERB":
            verbs.append(token.lemma_)
            nav.append(token.lemma_)

    sentences = []
    for sentence in doc.sents:
        sentences.append(str(sentence).strip())
        
    df.at[i, "sentences"] = "|".join(sentences)

    df.at[i, "nouns"] = "|".join(nouns)
    df.at[i, "adjectives"] = "|".join(adjectives)
    df.at[i, "verbs"] = "|".join(verbs)
    df.at[i, "lemmas"] = "|".join(lemmas)

    df.at[i, "entities"] = "|".join([str(e) for e in doc.ents])

In [ ]:
df.to_parquet("breastcancer-linguistics.parquet", compression="zstd")